In [ ]:
# To visualize the data, we will use the pymatviz package
# ! pip install pymatviz

In [130]:
import numpy as np
import pandas as pd
from ast import literal_eval

from pymatgen.core import Structure
from pymatviz import StructureWidget
from pymatviz import structure_3d

from symmetry import find_spg
from pyxtal.symmetry import Group

In [131]:
data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/csp/alex20s/csp-8c3cb/adam_bs_8000_lr_0.001_decay_0_clip_1_A_119_W_28_N_21_a_1_w_1_l_1_Nf_5_Kx_16_Kl_4_h0_256_l_16_H_16_k_64_m_64_e_32_drop_0.1_0.1//relaxed_structures_CsPbI3_200_ehull.csv')

#data = pd.read_csv('/home/user_wanglei/private/datafile/crystalgpt/csp/alex20s/csp-07d3f/adam_bs_8000_lr_0.0001_decay_0_clip_1_A_119_W_28_N_21_a_1_w_1_l_1_Nf_5_Kx_16_Kl_4_h0_256_l_16_H_16_k_64_m_64_e_32_drop_0.1_0.1//relaxed_structures_CsPbI3_4100_ehull.csv')

In [132]:
data = data.sort_values(by='relaxed_ehull', ascending=True).reset_index(drop=True)


In [133]:
cif_strings = data['relaxed_cif']   # string of pymatgen structure dict

In [134]:
structures =[Structure.from_dict(literal_eval(cif)) for cif in cif_strings]

In [136]:
for i, struct in enumerate(structures):
    
    sg_num = find_spg(struct)
    g = Group(sg_num)
    # Get ehull values if available
    try:
        unrelaxed_ehull = data.loc[i, 'unrelaxed_ehull'] if 'unrelaxed_ehull' in data.columns else None
    except Exception:
        unrelaxed_ehull = None
    try:
        relaxed_ehull = data.loc[i, 'relaxed_ehull'] if 'relaxed_ehull' in data.columns else None
    except Exception:
        relaxed_ehull = None

    reduced_formula = struct.composition.reduced_formula
    num_atoms = len(struct)
    
    print(f"{i}: {reduced_formula} {num_atoms}, {sg_num} {g.symbol}, {unrelaxed_ehull:.6f}, {relaxed_ehull:.6f}")


0: CsPbI3 20, 62 Pnma, 0.639795, 0.002972
1: CsPbI3 20, 62 Pnma, 0.156483, 0.003084
2: CsPbI3 10, 63 Cmcm, 0.064098, 0.009853
3: CsPbI3 10, 63 Cmcm, 0.070515, 0.010271
4: CsPbI3 20, 62 Pnma, 2.662368, 0.015636
5: CsPbI3 10, 161 R3c, 2.527180, 0.019631
6: CsPbI3 10, 161 R3c, 0.238453, 0.019634
7: CsPbI3 10, 161 R3c, 1.906707, 0.021976
8: CsPbI3 10, 161 R3c, 4.128099, 0.022317
9: CsPbI3 30, 1 P1, 29.301458, 0.022736
10: CsPbI3 10, 194 P63/mmc, 0.163547, 0.023587
11: CsPbI3 10, 161 R3c, 0.471613, 0.023836
12: CsPbI3 10, 148 R-3, 0.195488, 0.025814
13: CsPbI3 10, 148 R-3, 0.110133, 0.026521
14: CsPbI3 10, 148 R-3, 0.310242, 0.026689
15: CsPbI3 10, 148 R-3, 0.043485, 0.026931
16: CsPbI3 10, 148 R-3, 0.080953, 0.027365
17: CsPbI3 10, 148 R-3, 0.476487, 0.027495
18: CsPbI3 10, 148 R-3, 0.232968, 0.027717
19: CsPbI3 10, 148 R-3, 0.072291, 0.027823
20: CsPbI3 10, 161 R3c, 0.401199, 0.031549
21: CsPbI3 5, 123 P4/mmm, 0.047648, 0.031799
22: CsPbI3 5, 123 P4/mmm, 0.042953, 0.031804
23: CsPbI3 5, 2

spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.
spglib: ssm_get_exact_positions failed.
spglib: get_bravais_exact_positions_and_lattice failed.


104: CsPbI3 20, 1 P1, 7.323213, 0.107293
105: CsPbI3 10, 2 P-1, 2.207734, 0.107308
106: CsPbI3 10, 1 P1, 2.495479, 0.107329
107: CsPbI3 10, 2 P-1, 3.033876, 0.107718
108: Cs(PbI6)2 30, 1 P1, 20.721744, 0.109387
109: CsI5 12, 1 P1, 6.543205, 0.110279
110: CsPbI3 10, 1 P1, 1.393329, 0.110941
111: CsPbI3 20, 1 P1, 10.455758, 0.112239
112: PbI4 10, 1 P1, 9.134119, 0.112542
113: CsPbI3 20, 4 P21, 7.294419, 0.113553
114: CsPbI3 10, 1 P1, 1.105644, 0.114336
115: CsPbI3 5, 1 P1, 0.857904, 0.114483
116: CsPbI3 5, 1 P1, 1.208028, 0.114667
117: CsPbI3 5, 1 P1, 1.142153, 0.118089
118: CsPbI3 5, 1 P1, 37.459852, 0.118103
119: CsPbI3 10, 7 Pc, 0.838537, 0.118201
120: CsPbI3 20, 7 Pc, 45.451608, 0.118542
121: CsI2 30, 1 P1, 5.760379, 0.121856
122: CsPbI3 20, 1 P1, 34.412023, 0.122386
123: CsPbI3 10, 1 P1, 11.983923, 0.122980
124: Cs2I3 5, 139 I4/mmm, 0.235620, 0.123291
125: Cs2I3 5, 12 C2/m, 1.169517, 0.124923
126: Cs2I3 5, 71 Immm, 0.500308, 0.127949
127: CsPbI3 10, 1 P1, 1.545433, 0.133857
128: CsP

In [123]:
structures[0]

Structure Summary
Lattice
    abc : 10.779234131166449 4.95326941318773 17.94414187034688
 angles : 89.9977723264344 89.99991505966322 89.99926000075247
 volume : 958.081709711027
      A : np.float64(10.779234130939217) np.float64(6.953750057600148e-05) np.float64(7.955259762106017e-06)
      B : np.float64(3.201965427625384e-05) np.float64(4.953269412116543) np.float64(9.791062360878305e-05)
      C : np.float64(1.3356687600552516e-05) np.float64(0.0003429729398889461) np.float64(17.944141867064225)
    pbc : True True True
PeriodicSite: Pb (1.71, 1.238, 1.162) [0.1586, 0.25, 0.06477]
PeriodicSite: Pb (3.68, 3.715, 10.13) [0.3414, 0.75, 0.5648]
PeriodicSite: Pb (9.069, 3.716, 16.78) [0.8413, 0.7502, 0.9352]
PeriodicSite: Pb (7.1, 1.239, 7.809) [0.6587, 0.2501, 0.4352]
PeriodicSite: I (1.836, 1.238, 8.993) [0.1703, 0.25, 0.5011]
PeriodicSite: I (3.555, 3.715, 17.96) [0.3298, 0.7499, 1.001]
PeriodicSite: I (8.944, 3.715, 8.952) [0.8297, 0.75, 0.4989]
PeriodicSite: I (7.225, 1.239, -0.0

In [79]:

structure_3d(structures[:10], n_cols=1)

In [ ]:
structures[14]

In [124]:
from pyxtal import pyxtal
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

crystal = structures[0]
spga = SpacegroupAnalyzer(crystal, symprec=tol)
crystal = spga.get_refined_structure()
c = pyxtal()
c.from_seed(crystal, tol=tol)


In [125]:
c


------Crystal from Seed------
Dimension: 3
Composition: Cs4Pb4I12
Group: P n m a (62)
 10.7792,   4.9533,  17.9441,  90.0000,  90.0000,  90.0000, orthorhombic
Wyckoff sites:
	Cs @ [ 0.0702  0.2500  0.8297], WP [4c] Site [.m.]
	Pb @ [ 0.3414  0.2500  0.0648], WP [4c] Site [.m.]
	 I @ [ 0.3297  0.2500  0.5012], WP [4c] Site [.m.]
	 I @ [ 0.9683  0.2500  0.6096], WP [4c] Site [.m.]
	 I @ [ 0.7095  0.2500  0.2808], WP [4c] Site [.m.]

In [126]:
structures[14]

Structure Summary
Lattice
    abc : 8.742536719191499 8.742410169369409 7.681688556900135
 angles : 89.99779871305982 90.00169650859837 119.99984941990691
 volume : 508.4598080966223
      A : np.float64(8.742536718058354) np.float64(4.80205361234718e-05) np.float64(-0.00013231465832619515)
      B : np.float64(-4.371226771087864) np.float64(7.571136774328897) np.float64(0.00017211349359533122)
      C : np.float64(-0.0001111935903563533) np.float64(0.00010196053922051289) np.float64(7.681688555418693)
    pbc : True True True
PeriodicSite: Pb (0.0003782, -0.0002556, -0.0002922) [2.638e-05, -3.377e-05, -3.804e-05]
PeriodicSite: Pb (-0.0002438, 4.144e-05, 3.84) [-2.215e-05, -1.259e-06, 0.4999]
PeriodicSite: Cs (0.001138, 5.046, 1.921) [0.3334, 0.6665, 0.25]
PeriodicSite: Cs (4.372, 2.523, 5.762) [0.6667, 0.3333, 0.75]
PeriodicSite: I (4.371, 4.946, 1.92) [0.8265, 0.6532, 0.25]
PeriodicSite: I (2.273, 1.314, 1.921) [0.3467, 0.1735, 0.25]
PeriodicSite: I (6.469, 1.313, 1.92) [0.8267, 0.17

In [ ]:
cif_string="""# generated using pymatgen
data_TiO2
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   3.78253968
_cell_length_b   3.78253968
_cell_length_c   9.61502157
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   TiO2
_chemical_formula_sum   'Ti4 O8'
_cell_volume   137.56794451
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_type_symbol
 _atom_type_oxidation_number
  Ti4+  4.0
  O2-  -2.0
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Ti4+  Ti0  1  0.50000000  0.50000000  0.50000000  1
  Ti4+  Ti1  1  0.50000000  0.00000000  0.75000000  1
  Ti4+  Ti2  1  0.00000000  0.00000000  0.00000000  1
  Ti4+  Ti3  1  0.00000000  0.50000000  0.25000000  1
  O2-  O4  1  0.00000000  0.50000000  0.45715213  1
  O2-  O5  1  0.50000000  0.50000000  0.70715213  1
  O2-  O6  1  0.50000000  0.00000000  0.54284787  1
  O2-  O7  1  0.00000000  0.00000000  0.79284788  1
  O2-  O8  1  0.50000000  0.00000000  0.95715213  1
  O2-  O9  1  0.00000000  0.00000000  0.20715212  1
  O2-  O10  1  0.00000000  0.50000000  0.04284788  1
  O2-  O11  1  0.50000000  0.50000000  0.29284788  1"""

In [ ]:
from pymatgen.io.cif import CifParser
parser = CifParser.from_str(cif_string)
crystal = parser.get_structures()[0]

In [ ]:
spga = SpacegroupAnalyzer(crystal, symprec=0.01)
c = pyxtal()
c.from_seed(crystal, tol=0.01)

In [ ]:
c